# Bitcoin (BTC) Price Analysis

This Jupyter Notebook is designed for testing purposes and provides a comprehensive analysis of Bitcoin's (BTC) historical price performance. 

## Libraries Used

- **Python**: The primary programming language used.
- **pandas**: For data manipulation and analysis.
- **matplotlib**: For creating static, animated, and interactive visualizations.
- **seaborn**: For making statistical graphics.
- **yfinance**: To fetch historical market data from Yahoo Finance.

## Install Required Libraries

We will need `pandas` for data manipulation, `matplotlib` and `seaborn` for visualization, and `yfinance` to download the Bitcoin price data. Insert and run the following code cell at the top of your notebook to install the necessary Python packages:

In [ ]:
%pip install pandas matplotlib seaborn yfinance ipykernel

## Download BTC Price Data
`yfinance` allows us to download historical market data from Yahoo Finance. We'll use it to download the daily time series of Bitcoin prices from 2009 until today. Note that Bitcoin data on platforms like Yahoo Finance may not be available from 2009 because it wasn't traded on standard exchanges until around 2010 or later.

In [ ]:
import yfinance as yf

# Download BTC price data
btc = yf.download('BTC-USD', start='2010-07-17') # Adjust the start date if earlier data becomes available

# Display the first few rows to check the beginning of the data
btc.head()

# Display the last few rows to ensure it's loaded
# btc.tail()

## Data Preprocessing

Download BTC price data using the yfinance API. We will calculate yearly profit/loss percentages and actual price changes in dollars.

In [ ]:
import pandas as pd
import yfinance as yf

# Download BTC price data
btc = yf.download('BTC-USD', start='2010-07-17')

# Create an empty DataFrame for the summary
btc_december_summary = pd.DataFrame()

# Loop through the years of interest
for year in range(2014, 2024):
    # Extract data for December of the specific year
    december_data = btc[(btc.index.month == 12) & (btc.index.year == year)]

    # Check if there is data for December of that year
    if not december_data.empty:
        # Get the last day of December or the closest available date
        last_day = december_data.last('D')

        # Prepare the data to be added to the summary
        summary_data = {
            'Date': last_day.index[-1].strftime('%Y-%m-%d'),
            'Closing Price': last_day['Close'].iloc[-1],
            'High Price': december_data['High'].max()
        }

        # Convert summary_data to DataFrame for the current year
        df_current_year = pd.DataFrame([summary_data])

        # Calculate the percentage and actual price change from the previous December 31st
        if year > 2014:  # Ensure there is a previous year to compare with
            previous_year_close = btc_december_summary.loc[btc_december_summary['Date'].str.startswith(str(year-1)), 'Closing Price'].values[0]
            current_year_close = summary_data['Closing Price']
            price_change = current_year_close - previous_year_close
            percent_change = (price_change / previous_year_close) * 100

            # Add the calculated values to the DataFrame for the current year
            df_current_year['Price Change ($)'] = price_change
            df_current_year['Percent Change (%)'] = percent_change
        else:
            # For the first year in the range, set changes to NaN as there is no previous year to compare
            df_current_year['Price Change ($)'] = pd.NA
            df_current_year['Percent Change (%)'] = pd.NA

        # Append the data for the year to the summary DataFrame
        btc_december_summary = pd.concat([btc_december_summary, df_current_year], ignore_index=True)

# Set Date column as index
btc_december_summary.set_index('Date', inplace=True)

btc_december_summary


## Data Preprocessing 2010 - 2023

Download BTC price data using the CryptoCompare API. We will calculate yearly profit/loss percentages and actual price changes in dollars.

In [ ]:
%pip install requests

In [ ]:
import requests
import pandas as pd

# Base URL for the CryptoCompare API
base_url = 'https://min-api.cryptocompare.com/data/v2/histoday'

# Initialize an empty list to store the yearly data
yearly_data = []

for year in range(2010, 2024):
    # The 'toTs' parameter is set to December 31st of the current year
    # The 'limit' parameter is used to fetch the last record (0 means only the last day's data)
    params = {
        'fsym': 'BTC',
        'tsym': 'USD',
        'limit': 1,
        'toTs': pd.Timestamp(year=year, month=12, day=31).timestamp()
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the closing price and the high price from the response
    if data['Response'] == 'Success':
        price_info = data['Data']['Data'][-1]  # Get the last (or only) record
        date = pd.to_datetime(price_info['time'], unit='s').strftime('%Y-%m-%d')
        close_price = price_info['close']
        high_price = price_info['high']
        low_price = price_info['low']

        # Append the data for the current year to the list
        yearly_data.append([date, close_price, high_price, low_price])

# Convert the list to a DataFrame
btc_december_summary = pd.DataFrame(yearly_data, columns=['Date', 'Closing Price', 'High Price', 'Low Price'])

# Calculate the price change and percentage change from the previous year
btc_december_summary['Price Change ($)'] = btc_december_summary['Closing Price'].diff()
btc_december_summary['Percentage Change (%)'] = btc_december_summary['Closing Price'].pct_change() * 100

# Display the DataFrame
btc_december_summary


## Chart BTC Percentage Change

Visualize the actual annual percentage change of Bitcoin's closing price on December 31st for each year from 2010 to 2023

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Base URL for the CryptoCompare API
base_url = 'https://min-api.cryptocompare.com/data/v2/histoday'

# Initialize an empty list to store the yearly data
yearly_data = []

for year in range(2010, 2024):
    # The 'toTs' parameter is set to December 31st of the current year
    # The 'limit' parameter is used to fetch the last record (0 means only the last day's data)
    params = {
        'fsym': 'BTC',
        'tsym': 'USD',
        'limit': 1,
        'toTs': pd.Timestamp(year=year, month=12, day=31).timestamp()
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the closing price and the high price from the response
    if data['Response'] == 'Success':
        price_info = data['Data']['Data'][-1]  # Get the last (or only) record
        date = pd.to_datetime(price_info['time'], unit='s').strftime('%Y-%m-%d')
        close_price = price_info['close']
        high_price = price_info['high']

        # Append the data for the current year to the list
        yearly_data.append([date, close_price, high_price])

# Convert the list to a DataFrame
btc_december_summary = pd.DataFrame(yearly_data, columns=['Date', 'Closing Price', 'High Price'])

# Calculate the price change and percentage change from the previous year
btc_december_summary['Price Change ($)'] = btc_december_summary['Closing Price'].diff()
btc_december_summary['Percentage Change (%)'] = btc_december_summary['Closing Price'].pct_change() * 100

# Plotting
plt.figure(figsize=(14, 7))
plt.plot(btc_december_summary['Date'], btc_december_summary['Percentage Change (%)'], marker='o', linestyle='-', color='b')
plt.title('Annual Percentage Change of BTC Closing Price on December 31st')
plt.xlabel('Year')
plt.ylabel('Percentage Change (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

fig, ax1 = plt.subplots(figsize=(14, 7))

color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Profit/Loss (%)', color=color)
ax1.plot(yearly_summary.index, yearly_summary['Profit/Loss (%)'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Price Change ($)', color=color)
ax2.plot(yearly_summary.index, yearly_summary['Price Change ($)'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('BTC Yearly Profit/Loss Percentage and Price Change')
plt.show()

## Chart ETH Percentage Change

Visualize the actual annual percentage change of Ethereum's closing price on December 31st for each year from 2015 to 2023

In [ ]:
import requests
import pandas as pd

# Base URL for the CryptoCompare API
base_url = 'https://min-api.cryptocompare.com/data/v2/histoday'

# Initialize an empty list to store the yearly data
yearly_data = []

for year in range(2015, 2024):  # Ethereum was launched in 2015
    # The 'toTs' parameter is set to December 31st of the current year
    # The 'limit' parameter is used to fetch the last record (0 means only the last day's data)
    params = {
        'fsym': 'ETH',  # Changed from BTC to ETH
        'tsym': 'USD',
        'limit': 1,
        'toTs': pd.Timestamp(year=year, month=12, day=31).timestamp()
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the closing price, the high price, and the low price from the response
    if data['Response'] == 'Success':
        price_info = data['Data']['Data'][-1]  # Get the last (or only) record
        date = pd.to_datetime(price_info['time'], unit='s').strftime('%Y-%m-%d')
        close_price = price_info['close']
        high_price = price_info['high']
        low_price = price_info['low']

        # Append the data for the current year to the list
        yearly_data.append([date, close_price, high_price, low_price])

# Convert the list to a DataFrame
eth_december_summary = pd.DataFrame(yearly_data, columns=['Date', 'Closing Price', 'High Price', 'Low Price'])

# Calculate the price change and percentage change from the previous year
eth_december_summary['Price Change ($)'] = eth_december_summary['Closing Price'].diff()
eth_december_summary['Percentage Change (%)'] = eth_december_summary['Closing Price'].pct_change() * 100

# Display the DataFrame
eth_december_summary


## Chart SOL Percentage Change

Visualize the actual annual percentage change of Solana's closing price on December 31st for each year from 2015 to 2023

In [ ]:
import requests
import pandas as pd

# Base URL for the CryptoCompare API
base_url = 'https://min-api.cryptocompare.com/data/v2/histoday'

# Initialize an empty list to store the yearly data
yearly_data = []

for year in range(2020, 2024):  # Solana was launched in 2020
    # The 'toTs' parameter is set to December 31st of the current year
    # The 'limit' parameter is used to fetch the last record (0 means only the last day's data)
    params = {
        'fsym': 'SOL',  # Changed from ETH to SOL
        'tsym': 'USD',
        'limit': 1,
        'toTs': pd.Timestamp(year=year, month=12, day=31).timestamp()
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the closing price, the high price, and the low price from the response
    if data['Response'] == 'Success':
        price_info = data['Data']['Data'][-1]  # Get the last (or only) record
        date = pd.to_datetime(price_info['time'], unit='s').strftime('%Y-%m-%d')
        close_price = price_info['close']
        high_price = price_info['high']
        low_price = price_info['low']

        # Append the data for the current year to the list
        yearly_data.append([date, close_price, high_price, low_price])

# Convert the list to a DataFrame
sol_december_summary = pd.DataFrame(yearly_data, columns=['Date', 'Closing Price', 'High Price', 'Low Price'])

# Calculate the price change and percentage change from the previous year
sol_december_summary['Price Change ($)'] = sol_december_summary['Closing Price'].diff()
sol_december_summary['Percentage Change (%)'] = sol_december_summary['Closing Price'].pct_change() * 100

# Display the DataFrame
sol_december_summary
